<a href="https://colab.research.google.com/github/khalilouardini/IDRiD-challenge/blob/master/train_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('drive/My Drive/IDRiD-challenge')

In [6]:
!ls

classifier_utils.py  IDRiDClassifier.py       Segmentation_task
datasets	     IDRIDetectionDataset.py  train_classifier.ipynb
detection.ipynb      __pycache__


In [0]:
from IDRiDClassifier import IDRiDClassifier
from datasets.IDRiDGradingDataset import IDRiDGradingDataset
import copy
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy
import numpy as np
import torch
import pandas as pd

In [13]:
# Prepare data
root_dir_train = "/content/drive/My Drive/B. Disease Grading/1. Original Images/a. Training Set"
csv_file_train = "/content/drive/My Drive/B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv"

root_dir_test = "/content/drive/My Drive/B. Disease Grading/1. Original Images/b. Testing Set"
csv_file_test = "/content/drive/My Drive/B. Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv"
bengraham = False

# transform (preprocessing)
scale = transforms.Resize((64, 64))
to_tensor = transforms.ToTensor()
composed = transforms.Compose([scale,
                               to_tensor])
# train and test set
train_set = IDRiDGradingDataset(csv_file_train, root_dir_train, composed, bengraham)
test_set = IDRiDGradingDataset(csv_file_test, root_dir_test, composed, bengraham)

#dataloaders
batch_size = 4
train_loader = DataLoader(train_set, batch_size=batch_size,
                            shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=batch_size,
                            shuffle=True, num_workers=0)

print(f"# batches in train set {len(train_loader)} | # batches in test set {len(test_loader)}")

# batches in train set 104 | # batches in test set 26


In [14]:
# model
use_cuda = torch.cuda.is_available()
model = IDRiDClassifier()
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    model = torch.nn.DataParallel(model)
model.to(device)

# intialize variables
n_epochs = 20
lr = 1e-4
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
import numpy as np
import torch
import pandas as pd
from copy import deepcopy
import pdb

def train(model, train_loader, criterion, optimizer, n_epochs, device):
    """
    Method used to train our classifier

    Args:
        model: (nn.Module) the neural network
        train_loader: (DataLoader) a DataLoader wrapping a MRIDataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
        optimizer: (torch.optim) an optimization algorithm
        n_epochs: (int) number of epochs performed during training

    Returns:
        best_model: (nn.Module) the trained neural network
    """
    best_model = deepcopy(model)
    train_best_loss = np.inf

    for epoch in range(n_epochs):
        model.train()
        for i, data in enumerate(train_loader, 0):
            # load batch
            images, labels = data['image'], data['labels']
            images = images.to(device)
            labels = labels.to(device)
            # - forward pass
            output = model.forward(images)
            # - loss computation
            loss = criterion(output, labels)
            # - backward pass (gradients computation)
            loss.backward()
            # - weights update
            optimizer.step()
            # - gradients set to 0
            optimizer.zero_grad()

        _, train_metrics = test(model, train_loader, criterion, device)

        print('Epoch %i: loss = %f, balanced accuracy = %f'
              % (epoch, train_metrics['mean_loss'],
                 train_metrics['balanced_accuracy']))

        if train_metrics['mean_loss'] < train_best_loss:
            best_model = deepcopy(model)
            train_best_loss = train_metrics['mean_loss']

    return best_model


def test(model, data_loader, criterion, device):
    """
    Method used to test a CNN

    Args:
        model: (nn.Module) the model
        data_loader: (DataLoader) a DataLoader
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images

    Returns:
        results_df: (DataFrame) the label predicted for every subject
        results_metrics: (dict) a set of metrics
    """
    model.eval()
    columns = ["img_idx", "proba",
               "true_label", "predicted_label"]
    results_df = pd.DataFrame(columns=columns)
    total_loss = 0

    with torch.no_grad():
        for i, data in enumerate(data_loader, 0):
            images, labels = data['image'].to(device), data['labels'].to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            proba = torch.nn.Softmax(dim=1)(outputs)
            predictions, predicted_labels = torch.max(outputs.data, dim=1).values, torch.max(outputs.data, dim=1).indices
            for idx, _ in enumerate(labels):
                row = [idx, predictions[idx], labels[idx], predicted_labels[idx]]
                row_df = pd.DataFrame([row], columns=columns)
                results_df = pd.concat([results_df, row_df])

    results_metrics = compute_metrics(results_df.true_label.values, results_df.predicted_label.values)
    results_df.reset_index(inplace=True, drop=True)
    results_metrics['mean_loss'] = total_loss / len(data_loader.dataset)

    return results_df, results_metrics



def compute_metrics(ground_truth, prediction):
    """Computes the accuracy, sensitivity, specificity and balanced accuracy"""
    tp = np.sum((prediction == 1) & (ground_truth == 1))
    tn = np.sum((prediction == 0) & (ground_truth == 0))
    fp = np.sum((prediction == 1) & (ground_truth == 0))
    fn = np.sum((prediction == 0) & (ground_truth == 1))

    metrics_dict = dict()
    metrics_dict['accuracy'] = (tp + tn) / (tp + tn + fp + fn)

    # Sensitivity
    if tp + fn != 0:
        metrics_dict['sensitivity'] = tp / (tp + fn)
    else:
        metrics_dict['sensitivity'] = 0.0

    # Specificity
    if fp + tn != 0:
        metrics_dict['specificity'] = tn / (fp + tn)
    else:
        metrics_dict['specificity'] = 0.0

    metrics_dict['balanced_accuracy'] = (metrics_dict['sensitivity'] + metrics_dict['specificity']) / 2

    return metrics_dict

In [0]:
from classifier_utils import train, test

In [28]:
best_model = train(model, train_loader, criterion, optimizer, n_epochs, device)

Epoch 0: loss = 1.847604, balanced accuracy = 0.472826


In [32]:
results_df, results_metrics = test(model, test_loader, criterion, device)
print(results_metrics)
print("done")

    img_idx  ...             predicted_label
0         0  ...  tensor(0, device='cuda:0')
1         1  ...  tensor(2, device='cuda:0')
2         2  ...  tensor(2, device='cuda:0')
3         3  ...  tensor(0, device='cuda:0')
4         0  ...  tensor(0, device='cuda:0')
..      ...  ...                         ...
98        2  ...  tensor(0, device='cuda:0')
99        3  ...  tensor(0, device='cuda:0')
100       0  ...  tensor(1, device='cuda:0')
101       1  ...  tensor(2, device='cuda:0')
102       2  ...  tensor(0, device='cuda:0')

[103 rows x 4 columns] {'accuracy': 0.9130434782608695, 'sensitivity': 0.0, 'specificity': 0.9545454545454546, 'balanced_accuracy': 0.4772727272727273, 'mean_loss': 2.0546498669003976}
done
